## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-29-16-55-03 +0000,startribune,'Worst-case scenario of famine' is happening i...,https://www.startribune.com/food-crisis-expert...
1,2025-07-29-16-54-33 +0000,wapo,Ciara gains Beninese citizenship as country re...,https://www.washingtonpost.com/world/2025/07/2...
2,2025-07-29-16-53-04 +0000,nyt,UnitedHealth’s Profits Fall as Medical Care Co...,https://www.nytimes.com/2025/07/29/health/unit...
3,2025-07-29-16-52-56 +0000,bbc,Trump discusses whisky tariffs and Gaza with S...,https://www.bbc.com/news/articles/cx2q5pwzz1yo...
4,2025-07-29-16-50-13 +0000,nypost,"Trump confirms possible China trip, but insist...",https://nypost.com/2025/07/29/us-news/trump-co...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,65
5,gaza,22
81,new,19
80,killed,17
99,deal,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
151,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...,131
277,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,122
99,2025-07-29-13-11-06 +0000,nypost,President Trump mourns victims of NYC Midtown ...,https://nypost.com/2025/07/29/us-news/trump-mo...,113
260,2025-07-28-19-58-00 +0000,wsj,The Trump administration is considering a plan...,https://www.wsj.com/politics/policy/patent-sys...,112
131,2025-07-29-11-00-00 +0000,wsj,Restructured deduction in President Trump’s ne...,https://www.wsj.com/business/u-s-exporters-get...,110


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
151,131,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...
16,78,2025-07-29-16-32-40 +0000,wapo,NYC shooting live updates: Police investigate ...,https://www.washingtonpost.com/nation/2025/07/...
52,66,2025-07-29-15-30-51 +0000,nypost,Union Pacific strikes $85B deal to buy rival N...,https://nypost.com/2025/07/29/business/union-p...
15,51,2025-07-29-16-32-59 +0000,nyt,UK Will Recognize Palestinian Statehood In Sep...,https://www.nytimes.com/2025/07/29/world/europ...
110,45,2025-07-29-12-22-17 +0000,nypost,NYC gunman Shane Tamura referenced ex-NFL play...,https://nypost.com/2025/07/29/us-news/nyc-gunm...
0,44,2025-07-29-16-55-03 +0000,startribune,'Worst-case scenario of famine' is happening i...,https://www.startribune.com/food-crisis-expert...
234,37,2025-07-28-21-38-24 +0000,nypost,Bill Maher admits he was wrong about Trump on ...,https://nypost.com/2025/07/28/media/bill-maher...
152,31,2025-07-29-09-24-00 +0000,wsj,Europe’s Star Economy Picks Up Pace as Fresh T...,https://www.wsj.com/world/europe/europes-star-...
94,30,2025-07-29-13-27-59 +0000,latimes,"New York gunman, an L.A. high school football ...",https://www.latimes.com/california/story/2025-...
153,30,2025-07-29-09-22-09 +0000,bbc,Beijing floods kill 30 as China sees summer of...,https://www.bbc.com/news/articles/cg7j8x3xnrko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
